In [1]:
import gc
import re
import time
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

from pathlib import Path
import os
import socket
import numpy as np
import torch
from datasets import load_dataset
from deepspeed import DeepSpeedEngine
import deepspeed
from tqdm.notebook import tqdm
from dataclasses import dataclass, asdict

from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedModel
from vllm import LLM, SamplingParams
from vllm.inputs import TokensPrompt

import wandb

def find_free_port():
    """Find a free port on localhost."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        s.listen(1)
        port = s.getsockname()[1]
    return port

# Needed to stop DeepSpeed from complaining
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = str(find_free_port())
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

SCRATCH = Path.home() / "workspace" / "scratch"
os.environ["HF_HOME"] = str(SCRATCH / "hf_home")

RUN_NAME = "r1-zero"
EXP_DIR = SCRATCH / "deepseek_r1z_hackathon" / RUN_NAME
EXP_DIR.mkdir(parents=True, exist_ok=True)
print(f"Logs and Checkpoints will be saved to: {EXP_DIR}")

os.environ["VLLM_USE_V1"] = "0"

DATASET_NAME = "Jiayi-Pan/Countdown-Tasks-3to4"
# MODEL_NAME = "Qwen/Qwen2.5-1.5B"
MODEL_NAME = "Qwen/Qwen2.5-3B"
EOS_TOKEN = "<|endoftext|>" # has to match the model

SYSTEM_PROMPT = (
    "You are a helpful assistant. You first think about the reasoning process "
    "and then provide the user with the answer."
)

PROMPT_TEMPLATE = (
    "Using the numbers {numbers}, create an equation that equals {target}. "
    "You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. "
    "Show your work in <think> </think> tags. And return the final equation and answer in "
    "<answer> </answer> tags, for example <answer>(1 + 2) / (3 * 5)</answer>."
)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

[2025-09-17 23:07:01,664] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


INFO 09-17 23:07:05 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 09-17 23:07:05 [__init__.py:239] Automatically detected platform cuda.
Logs and Checkpoints will be saved to: /root/workspace/scratch/deepseek_r1z_hackathon/r1-zero


In [2]:
import re
from typing import Tuple
def format_reward_func(completion: str) -> float:
    """
    Format: <think>...</think>\n</answer>...</answer>

    Also checks that the content within <answer>...</answer> conforms to a
    specified pattern (only digits, + - * / ( ) . and whitespace).

    Args:
        completion (str): Generated output

    Returns:
        float: Reward score
    """
    # Define the allowed pattern (only numbers, +, -, *, /, (, ), ., and whitespace)
    allowed_pattern = r"^[\d+\-*/().\s]+$"

    try:
        # add synthetic <think> as its already part of the prompt and prefilled
        # for the assistant to more easily match the regex
        completion = "<think>" + completion

        # Strip EOS token if present
        if completion.endswith(EOS_TOKEN):
            completion = completion[:-len(EOS_TOKEN)]

        # Check if the format is correct
        # Pattern means:
        # 1) <think>...contents not including other <think> tags...</think>
        # 2) \n
        # 3) <answer>...anything...</answer>
        regex = r"^<think>([^<]*(?:<(?!/?think>)[^<]*)*)<\/think>\n<answer>([\s\S]*?)<\/answer>$"
        match = re.search(regex, completion, re.DOTALL)

        if match is None or len(match.groups()) != 2:
            # Format is incorrect
            return 0.0
        else:
            # Extract the content inside <answer>...</answer>
            answer_content = match.group(2).strip()

            # Check if answer content matches the allowed pattern
            if not re.match(allowed_pattern, answer_content):
                # If it doesn't match, reward is 0.5
                return 0.5
            else:
                # If both format and pattern are correct, reward is 1
                return 1.0
    except Exception:
        # Any error leads to 0 reward
        return 0.0


def equation_reward_func(completion: str, nums: List[int], target: int) -> float:
    """
    Evaluates completion based on mathematical correctness of the answer

    Args:
        completion (str): Generated output
        target (str): Expected answer
        nums (list): Available numbers to use in the equation

    Returns:
        float: Reward score
    """
    try:
        # Check if the format is correct
        match = re.search(r"<answer>(.*?)<\/answer>", completion)
        if match is None:
            return 0.0
        # Extract the "answer" part from the completion
        equation = match.group(1).strip()
        # Extract all numbers from the equation
        used_numbers = [int(n) for n in re.findall(r"\d+", equation)]

        # Check if all numbers are used exactly once
        if sorted(used_numbers) != sorted(nums):
            return 0.0
        # Define a regex pattern that only allows numbers, operators, parentheses, and whitespace
        allowed_pattern = r"^[\d+\-*/().\s]+$"
        if not re.match(allowed_pattern, equation):
            return 0.0

        # Evaluate the equation with restricted globals and locals
        result = eval(equation, {"__builtins__": None}, {})
        # Check if the equation is correct and matches the ground truth
        if abs(float(result) - float(target)) < 1e-5:
            return 1.0
        else:
            return 0.0
    except Exception:
        # If evaluation fails, reward is 0
        return 0.0


def compute_reward(completion: str, sample: Dict[str, Any]) -> Tuple[float, Dict[str, float]]:
    nums = sample["nums"]
    target = sample["target"]

    format_reward = format_reward_func(completion)
    equation_reward = equation_reward_func(
        completion=completion, nums=nums, target=target
    )

    reward = format_reward + equation_reward

    metrics = {
        "format_reward": format_reward,
        "equation_reward": equation_reward,
    }

    return reward, metrics

In [3]:
dataset = load_dataset(DATASET_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def process_sample(example: Dict[str, int]) -> Dict[str, int]:
    prefix = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": PROMPT_TEMPLATE.format(numbers=example["nums"], target=example["target"])},
        {"role": "assistant", "content": "Let me reason through this step by step.\n<think>"},
    ]

    input_ids = tokenizer.apply_chat_template(
        prefix, tokenize=True, continue_final_message=True
    )
    prompt = tokenizer.decode(
        input_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False
    )
    return {"prompt": prompt, "prompt_token_ids": input_ids}

dataset = dataset.map(process_sample, num_proc=6)
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

In [4]:
@dataclass
class AgentConfig():
    model_name: str = MODEL_NAME

    # hyperparameters for sampling
    gen_per_sample: int = 4
    temperature: float = 1.0
    max_new_tokens: int = 1024
    top_p: float = 1.0  # disable top_p
    top_k: int = -1     # disable tok_k
    num_rollouts: int = 16
    kl_coefficient: float = 0.001
    n_rollouts: int = 64
    batch_size: int = 4
    learning_rate: float = 1e-6

# Now you can use the model for inference
class Agent():
    def __init__(
            self,
            config: AgentConfig,
            tokenizer: AutoTokenizer,
            device: torch.device,
        ):

        self.model_name = config.model_name

        self.inference_engine = LLM(
            model=config.model_name,
            skip_tokenizer_init=False,
            gpu_memory_utilization=0.2,
            enable_prefix_caching=True,
            swap_space=4,
            scheduling_policy="fcfs",
            dtype=torch.bfloat16,
            max_model_len=config.max_new_tokens * 1.5,
            enable_sleep_mode=True,
        )
                
        self.inference_engine.sleep(1)

        # hyperparameters for sampling
        self.gen_per_sample = config.gen_per_sample
        self.temperature = config.temperature
        self.max_new_tokens = config.max_new_tokens
        self.top_k = config.top_k
        self.top_p = config.top_p
        self.num_rollouts = config.num_rollouts
        self.kl_coefficient = config.kl_coefficient
        self.n_rollouts = config.n_rollouts
        self.batch_size = config.batch_size
        self.learning_rate = config.learning_rate

        
        deepspeed_config = {
            "bf16": {"enabled": True},
            "zero_optimization": {"stage": 2, "overlap_comm": False},
            "train_batch_size": self.num_rollouts * self.gen_per_sample,
            "train_micro_batch_size_per_gpu": self.batch_size,
            "gradient_accumulation_steps": (self.num_rollouts * self.gen_per_sample)// self.batch_size,
            "gradient_clipping": 1.0,
            "optimizer": {
                "type": "AdamW",
                "params": {
                    "lr": self.learning_rate,
                    "betas": (0.9, 0.999),
                    "eps": 1e-8,
                    "weight_decay": 0.0,
                    "torch_adam": True,
                },
            },
        }
        # DeepSpeed config for the reference model
        ref_deepspeed_config = {
            "bf16": {"enabled": True},
            # Note that we don't train the reference model
            # These are just for compatibility with DeepSpeed.
            "train_batch_size": self.num_rollouts * self.gen_per_sample,

            "train_micro_batch_size_per_gpu": self.batch_size,
            "gradient_accumulation_steps": (self.num_rollouts * self.gen_per_sample) // self.batch_size,
        }
        
        self.policy = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            device_map=0,
        )
        self.reference = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            device_map=0,
        )
        
        self.policy.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

        self.policy, *_ = deepspeed.initialize(
            model=self.policy,
            config=deepspeed_config,
            model_parameters=self.policy.parameters(),
        )
        self.reference, *_ = deepspeed.initialize(
            model=self.reference,
            config=ref_deepspeed_config,
        )

        self.reference.module.cpu()
        
        self.tokenizer = tokenizer
        self.eos_token_id = tokenizer.eos_token_id
        self.pad_token_id = self.eos_token_id
        self.ignore_idx = -100


        self.device = device

        wandb.init(
            project="llm-rl",
            config=config
        )

    def expand_list(self, d_list, rep) -> List[Any]:
        return [item for item in d_list for _ in range(rep)]

    def rollout(
        self,
        samples: Dict[str, Any],
    ) -> Dict[str, torch.Tensor]:


        inputs = [TokensPrompt(prompt_token_ids=ids) for ids in samples["prompt_token_ids"]]

        sp = SamplingParams(
            n=self.gen_per_sample,
            temperature=self.temperature,
            top_p=self.top_p,
            top_k=self.top_k,
            max_tokens=self.max_new_tokens,
            stop_token_ids=[self.eos_token_id],
        )

        gc.collect()
        torch.cuda.empty_cache()
        time.sleep(1)

        self.inference_engine.wake_up()
        rollouts = self.inference_engine.generate(inputs, sampling_params=sp)
        self.inference_engine.sleep(1)

        gc.collect()
        torch.cuda.empty_cache()
        time.sleep(1)

        gen_token_ids = [out.token_ids for req in rollouts for out in req.outputs]
        finish_reasons = [out.finish_reason for req in rollouts for out in req.outputs]

        responses = self.tokenizer.batch_decode(gen_token_ids, skip_special_tokens=False)

        rollouts = {
            "nums": self.expand_list(samples["nums"], self.gen_per_sample),
            "target": self.expand_list(samples["target"], self.gen_per_sample),
            "prompt": self.expand_list(samples["prompt"], self.gen_per_sample),
            "prompt_token_ids": self.expand_list(samples["prompt_token_ids"], self.gen_per_sample),
            "gen_token_ids": gen_token_ids,
            "response": responses,
            "finish_reason": finish_reasons,
        }

        return rollouts

    def prep_model_inputs(
            self,
            rollouts: Dict[str, Any],
            pad_id: int = 0,
        ) -> Dict[str, Any]:
        # --- sanity checks ---
        assert len(rollouts["prompt_token_ids"]) == len(rollouts["gen_token_ids"]) \
            == len(rollouts["target"]) == len(rollouts["nums"]) \
            == len(rollouts["finish_reason"])
        num_rollouts = len(rollouts["prompt_token_ids"])

        # --- sequence lengths ---
        max_seq_len = max(
            len(p) + len(g)
            for p, g in zip(rollouts["prompt_token_ids"], rollouts["gen_token_ids"])
        )

        # --- rewards ---
        rewards_and_metrics = [
            compute_reward(resp, {"nums": nums, "target": tgt})
            for resp, nums, tgt in zip(rollouts["response"], rollouts["nums"], rollouts["target"])
        ]
        rewards, metrics = zip(*rewards_and_metrics)
        assert len(rewards) == num_rollouts

        rewards = np.array(rewards, dtype=np.float32)
        
        # for logging
        f_rewards = np.array([m["format_reward"] for m in metrics], dtype=np.float32)
        e_rewards = np.array([m["equation_reward"] for m in metrics], dtype=np.float32)
        finished_reasons = np.array(
            [1 if r == "stop" else 0 for r in rollouts["finish_reason"]],
            dtype=int
        )
    
        wandb.log({
            "mean_reward": np.mean(rewards),
            "mean_f_reward": np.mean(f_rewards),
            "mean_q_reward": np.mean(e_rewards),
            "frac_finished": np.sum(finished_reasons) / len(finished_reasons),
            "avg_gen_len": np.mean([len(g) for g in rollouts["gen_token_ids"]]),
        })
 
        # --- group-normalized advantages per rollout ---
        adv_chunks = []
        for i in range(0, num_rollouts, self.gen_per_sample):
            r_group = rewards[i:i + self.gen_per_sample]
            group_adv = (r_group - r_group.mean()) / (r_group.std() + 1e-4)
            adv_chunks.append(group_adv)
        per_rollout_adv = np.concatenate(adv_chunks, axis=0)  # shape: [num_rollouts]
        assert per_rollout_adv.shape[0] == num_rollouts

        device = self.device
        # === create output tensors ===
        input_token_ids = torch.full(
            (num_rollouts, max_seq_len),
            fill_value=pad_id,
            dtype=torch.long,
            device=device,
        )
        attn_mask = torch.zeros(
            (num_rollouts, max_seq_len),
            dtype=torch.bool,
            device=device,
        )
        labels = torch.full(
            (num_rollouts, max_seq_len),
            fill_value=self.ignore_idx,
            dtype=torch.long,
            device=device,
        )
        labels_mask = torch.zeros(
            (num_rollouts, max_seq_len),
            dtype=torch.bool,
            device=device,
        )
        advantages = torch.zeros(
            (num_rollouts, max_seq_len),
            dtype=torch.float32,
            device=device,
        )

        # --- fill rows ---
        for idx, (p_ids, g_ids, roll_adv) in enumerate(
            zip(rollouts["prompt_token_ids"], rollouts["gen_token_ids"], per_rollout_adv)
        ):
            g_ids = list(g_ids)
            seq = p_ids + g_ids
            seq_len = len(seq)
            p_len = len(p_ids)
            g_len = len(g_ids)

            # input ids
            input_token_ids[idx, :seq_len] = torch.tensor(seq, dtype=torch.long, device=device)
            # attention mask: attend to real tokens only
            attn_mask[idx, :seq_len] = True
            # labels: only the generated part counts; prompt part is padded/ignored
            labels[idx, p_len:p_len+g_len] = torch.tensor(g_ids, dtype=torch.long, device=device)
            labels_mask[idx, p_len:p_len+g_len] = True
            # per-token advantages on generated region (same scalar across the gen span)
            if g_len > 0:
                advantages[idx, p_len:p_len+g_len] = float(roll_adv)

        return {
            "input_token_ids": input_token_ids,
            "attn_mask": attn_mask,
            "advantages": advantages,
            "labels": labels,
            "labels_mask": labels_mask,
            "finish_reason": rollouts["finish_reason"]
        }
    
    def log_probs(self, model, model_inputs, temperature) -> torch.Tensor:
        outputs = model(
            input_ids=model_inputs["input_token_ids"],
            attention_mask=model_inputs["attn_mask"],
            return_dict=True,
            use_cache=False,
        )
        # temperature scaling (make sure temperature > 0)
        logits = outputs.logits / temperature
    
        # shift for next-token prediction
        shift_logits = logits[..., :-1, :]
        shift_labels = model_inputs["labels"][..., 1:]                # [B, T-1]
        shift_labels_mask = model_inputs["labels_mask"][..., 1:]      # [B, T-1]
    
        # log softmax over vocab
        vocab_log_probs = shift_logits.log_softmax(dim=-1)            # [B, T-1, V]
    
        # SAFELY gather: replace ignored labels with 0 (or any valid token id)
        safe_labels = torch.where(
            shift_labels_mask.bool(),
            shift_labels,
            torch.zeros_like(shift_labels)
        )
        token_logps = torch.gather(vocab_log_probs, dim=-1,
                                   index=safe_labels.unsqueeze(-1)).squeeze(-1)  # [B, T-1]
    
        # mask out ignored positions
        token_logps = token_logps * shift_labels_mask
        return token_logps  # [B, T-1] masked
    
    def compute_loss(self, batch: Dict[str, torch.Tensor], total_response_len: int):
        input_token_ids = batch["input_token_ids"]
        attention_mask  = batch["attn_mask"]
        labels          = batch["labels"]
        labels_mask     = batch["labels_mask"]
        advantages      = batch["advantages"]
    
        assert torch.all(labels_mask == (labels != self.ignore_idx))
    
        model_inputs = {
            "input_token_ids": input_token_ids,
            "attn_mask": attention_mask,
            "labels": labels,
            "labels_mask": labels_mask,
        }
    
        self.reference.to("cuda")
        with torch.no_grad():
            ref_logps = self.log_probs(self.reference, model_inputs, self.temperature)  # [B, T-1]
    
        logps = self.log_probs(self.policy, model_inputs, self.temperature)             # [B, T-1]
    
        # shift masks/advantages once to match [B, T-1]
        m = labels_mask[..., 1:]             # [B, T-1]
        adv = advantages[..., 1:]            # [B, T-1]
    
        # KL term (token-wise) using log-probs
        # d = ref_logps - logps;  exp(d) - d - 1 is a symmetric KL-like penalty
        d = ref_logps - logps
        kl_penalty = (torch.exp(d) - d - 1) * m
    
        # Entropy: negative mean log-prob on valid positions (optional: compute from full dist)
        entropy = -(logps.sum() / (m.sum().clamp_min(1)))
    
        # Policy loss (masked)
        policy_loss = -(logps * adv) * m
    
        loss = (policy_loss.sum() + self.kl_coefficient * kl_penalty.sum()) / max(total_response_len, 1)
    
        metrics = {
            "policy_loss": (policy_loss.sum().item() / max(total_response_len, 1)),
            "kl_penalty": (kl_penalty.sum().item() / max(total_response_len, 1)),
            "entropy": float(entropy.item()),
        }
        return loss, metrics
    
    def train(
        self,
        train_ds,
        test_ds=None,
        num_epochs: int = 1,
        eval_every: int = 0,
    ):
        """
        For each epoch:
          - Sample self.n_rollouts prompts
          - Generate self.gen_per_sample responses each (vLLM)
          - Build tensors
          - Accumulate grads over ALL mini-batches from this rollout
          - Do ONE optimizer step (and weight clip)
          - Refresh vLLM weights
        """
    
        history = {"loss": [], "policy_loss": [], "kl_penalty": [], "entropy": []}
    
        def _sample_batch(ds, n_items: int):
            import random
            if hasattr(ds, "select"):
                idxs = random.sample(range(len(ds)), n_items)
                return ds.select(idxs)
            else:
                idxs = random.sample(range(len(ds["prompt_token_ids"])), n_items)
                return {k: [ds[k][i] for i in idxs] for k in ds.keys()}
    
        global_step = 0
        for epoch in range(num_epochs):
            # 1) Sample prompts for this rollout batch
            batch_prompts = _sample_batch(train_ds, self.n_rollouts)
    
            # 2) Generate rollouts (expands rows by gen_per_sample)
            rollouts = self.rollout(batch_prompts)
    
            # 3) Pack tensors
            data = self.prep_model_inputs(rollouts, pad_id=self.pad_token_id)
            training_size = len(rollouts["prompt_token_ids"])  # = n_rollouts * gen_per_sample
            assert training_size == self.n_rollouts * self.gen_per_sample

            self.policy.to(self.device)
            self.policy.train()
            self.reference.to(self.device)
            self.reference.eval()
            
            # 4) Accumulate gradients over ALL mini-batches, then do ONE step
    
            accum_loss_value = 0.0
            accum_metrics = {"policy_loss": 0.0, "kl_penalty": 0.0, "entropy": 0.0}
    
            for b_start in tqdm(range(0, training_size, self.batch_size)):
                b_end = min(b_start + self.batch_size, training_size)
    
                batch = {
                    "input_token_ids": data["input_token_ids"][b_start:b_end],
                    "attn_mask":       data["attn_mask"][b_start:b_end],
                    "labels":          data["labels"][b_start:b_end],
                    "labels_mask":     data["labels_mask"][b_start:b_end],
                    "advantages":      data["advantages"][b_start:b_end],
                }
    
                # match shift in log_probs: count valid positions post-shift
                total_response_len = int(batch["labels_mask"][..., 1:].sum().item())
    
                loss, metrics = self.compute_loss(batch, total_response_len)

                # deepspeed backward
                self.policy.backward(loss, scale_wrt_gas=False)
                
                wandb.log({
                    "loss": loss.item(),
                    "policy_loss": metrics["policy_loss"],
                    "kl_penalty": metrics["kl_penalty"],
                    "entropy": metrics["entropy"],
                })
                

                accum_loss_value += float(loss.item())
                for k in accum_metrics:
                    accum_metrics[k] += metrics[k]

                del loss, metrics
               
                global_step += 1

                if self.policy.is_gradient_accumulation_boundary():
                    self.reference.module.cpu()

                self.policy.step()
                
                if eval_every and (global_step % eval_every == 0) and (test_ds is not None):
                    self.policy.eval()
                    # TODO: add quick eval
                    self.policy.train()
    
    
            # Log epoch-averaged metrics
            history["loss"].append(accum_loss_value)
            history["policy_loss"].append(accum_metrics["policy_loss"])
            history["kl_penalty"].append(accum_metrics["kl_penalty"])
            history["entropy"].append(accum_metrics["entropy"])

            print(f"[{epoch}/{num_epochs}]  loss: {accum_loss_value}, policy_loss: {accum_metrics['policy_loss']}, kl-penalty: {accum_metrics['kl_penalty']}, entropy:{accum_metrics['entropy']}")

            gc.collect()
            torch.cuda.empty_cache()
            time.sleep(1)
            # 5) Push updated weights into vLLM
            self.update_inference_engine()

            gc.collect()
            torch.cuda.empty_cache()
            time.sleep(1)
    
        return history

    def update_inference_engine(self) -> None:
        """
        Update the running vLLM inference engine with the latest weights
        from the training policy model.
        Assumes inference engine sleeps when called
        After call inference engine is ready to be used (i.e. awake)
    
        This function:
          1. Wake up vllm inference engine
          2. Copies the current HF model weights from self.policy.
          3. Pushes them into the vLLM engine without restarting.
    
        Returns:
            None
        """
        
        # --- wake the engine and push weights into the internal model ---
        self.inference_engine.wake_up()
        
        device = "cuda:0"
        
        # find the V0 path and call load_weights
        model_obj = getattr(
            getattr(
                getattr(self.inference_engine.llm_engine, "model_executor").driver_worker,
                "model_runner"
            ),
            "model"
        )
        state_dict = self.policy.module.state_dict() if isinstance(self.policy, DeepSpeedEngine) else model.state_dict()

        model_obj.load_weights(weights=state_dict.items())

In [ ]:
config = AgentConfig()
agent = Agent(config, tokenizer, DEVICE)
agent.update_inference_engine()
hist = agent.train(dataset["train"], dataset["test"], 1000, 0)

INFO 09-17 23:07:17 [config.py:717] This model supports multiple tasks: {'embed', 'classify', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
INFO 09-17 23:07:17 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5) with config: model='Qwen/Qwen2.5-3B', speculative_config=None, tokenizer='Qwen/Qwen2.5-3B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1536, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=None, served_model_name=Qw

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-17 23:07:20 [loader.py:458] Loading weights took 1.23 seconds
INFO 09-17 23:07:21 [model_runner.py:1140] Model loading took 5.7916 GiB and 1.569475 seconds
INFO 09-17 23:07:22 [worker.py:287] Memory profiling takes 0.70 seconds
INFO 09-17 23:07:22 [worker.py:287] the current vLLM instance can use total_gpu_memory (79.25GiB) x gpu_memory_utilization (0.20) = 15.85GiB
INFO 09-17 23:07:22 [worker.py:287] model weights take 5.79GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 8.57GiB.
INFO 09-17 23:07:22 [executor_base.py:112] # cuda blocks: 15608, # CPU blocks: 7281
INFO 09-17 23:07:22 [executor_base.py:117] Maximum concurrency for 1536 tokens per request: 162.58x
INFO 09-17 23:07:28 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 09-17 23:07:55 [model_runner.py:1592] Graph capturing finished in 26 secs, took 0.23 GiB
INFO 09-17 23:07:55 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 34.03 seconds
INFO 09-17 23:07:55 [prefix_caching_block.py:479] Successfully reset prefix cache
INFO 09-17 23:07:55 [prefix_caching_block.py:479] Successfully reset prefix cache
INFO 09-17 23:07:59 [worker.py:145] Sleep mode freed 14.44 GiB memory, 0.78 GiB memory is still in use.
INFO 09-17 23:07:59 [executor_base.py:210] It took 3.405036 seconds to fall asleep.


...

Processed prompts:   0%|          | 0/256 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

INFO 09-17 23:53:43 [prefix_caching_block.py:479] Successfully reset prefix cache
INFO 09-17 23:53:43 [prefix_caching_block.py:479] Successfully reset prefix cache
INFO 09-17 23:53:44 [worker.py:145] Sleep mode freed 15.32 GiB memory, 41.24 GiB memory is still in use.
INFO 09-17 23:53:44 [executor_base.py:210] It took 0.886988 seconds to fall asleep.


  0%|          | 0/64 [00:00<?, ?it/s]

[27/1000]  loss: -0.5930978904943913, policy_loss: -0.6028316898900647, kl-penalty: 9.732563124642063, entropy:34.84765625
INFO 09-17 23:55:09 [executor_base.py:226] It took 0.278431 seconds to wake up tags {'kv_cache', 'weights'}.
WARNING 09-17 23:55:12 [executor_base.py:215] Executor is not sleeping.


Processed prompts:   0%|          | 0/256 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]